In [1]:
import pandas as pd
import numpy as np
#get the smaller portion of data
table = pd.read_csv('processed_data.csv')
index = table[table["user_id"] == 36758].index[0]
table = table.iloc[:index]
utility_matrix = table.pivot_table(index='anime_id', columns='user_id', values='rating')
utility_matrix.fillna(0,inplace=True)
file_path = "50%_utility_matrix.csv"
utility_matrix.reset_index(inplace=True)
utility_matrix.to_csv(file_path,index=False)


In [2]:
utility_matrix.head(20)

user_id,anime_id,1,2,3,4,5,6,7,8,9,...,36748,36749,36750,36751,36752,36753,36754,36755,36756,36757
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,9.0,0.0,0.0,0.0,10.0,0.0,0.0,10.0
1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,0.0,0.0,0.0,-1.0,8.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0
3,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,15,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,17,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,18,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
#feature matrix X
import pandas as pd
feature_matrix = pd.read_csv("processed_data.csv")

feature_matrix = feature_matrix[['anime_id', 'genre_vector']].drop_duplicates()
feature_matrix = feature_matrix.sort_values(by='anime_id')
# Save the result into a CSV file
feature_matrix.to_csv("feature_matrix.csv", index=False)


In [ ]:
Y = pd.read_csv("50%_utility_matrix.csv")



In [15]:
def get_SubX_and_SubY(n:int):
    #user n
    column_n = str(n)
    sub_y = Y[Y[column_n]!=0][['anime_id',column_n]]
    merge_table = pd.merge(feature_matrix,sub_y,on =  'anime_id') 
    subY = merge_table[column_n].values
    subX = merge_table['genre_vector'].values
    del sub_y,merge_table
    total,N = 0,0
    for rating in subY:
        if rating != -1:
            total += rating
            N+=1
    #for i in range(len(subY)):
        #if subY[i] == -1:
            #subY[i] = round(total/N,2)
    return subX,subY

SubX,SubY = get_SubX_and_SubY(1)
print(SubY)

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. 10. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. 10. -1. 10. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. 10. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1.]


: 